#Importing Python Libraries



---



In [ ]:
!pip install transformers
!pip install pycaret
!pip install numpy 
!pip install pandas
!pip install tensorflow
!pip install kaggle
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 6.1 MB/s eta 0:00:00

## Importing Dataset from Kaggle ( User needs a API Key)

In [ ]:
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

 93% 38.0M/41.0M [00:00<00:00, 127MB/s] 
100% 41.0M/41.0M [00:00<00:00, 118MB/s]


In [ ]:
!unzip fake-and-real-news-dataset.zip

Archive:  fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


# Preprocessing The Data from the dataset



---



In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer

fake_news = pd.read_csv('Fake.csv')
fake_news = fake_news.drop(["subject","date"],axis=1)
fake_news['label'] = 'fake'


true_news = pd.read_csv('True.csv')
true_news = true_news.drop(["subject","date"],axis=1)
true_news['label'] = 'true'


data = pd.concat([fake_news, true_news], axis=0)

# Shuffle the data
data = data.sample(frac=1).reset_index(drop=True)

data

,title,text,label
0,"Kim Davis Goes Off On Gay People, SCOTUS And ...",Just when you thought Kentucky County Clerk Ki...,fake
1,SEARING WaPo Op-Ed CRUSHES Rudy Giuliani’s Cl...,The rumors that Hillary Clinton is sick have b...,fake
2,Bombs kill Pakistani soldiers hunting U.S.-Can...,ISLAMABAD (Reuters) - Bomb blasts killed a Pak...,true
3,Turkey chides Arabs for 'weak' reaction ahead ...,ANKARA/ISTANBUL (Reuters) - Turkey criticized ...,true
4,WHY DEMOCRATS KEEP LOSING: Rep. Hakeem Jeffrie...,Hakeem Jeffries just exposed his Trump Derange...,fake
...,...,...,...
44893,Trump replaces chief of staff Priebus with ret...,WASHINGTON (Reuters) - President Donald Trump ...,true
44894,Senators press Trump for details on Icahn's sp...,(Reuters) - Six U.S. Senate Democrats on Thurs...,true
44895,Catalan leader Puigdemont to address Catalan p...,BARCELONA (Reuters) - Catalan leader Carles Pu...,true
44896,"Skirting Kurdish issue, France says Iraq's Aba...",PARIS (Reuters) - France appeared to backtrack...,true


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Create input sequences
input_ids = []
attention_masks = []
token_type_ids = []



for text in data['title']:
    encoded_dict = tokenizer.encode_plus(
                        text,                      
                        add_special_tokens = True, 
                        max_length = 256,          
                        pad_to_max_length = True,
                        return_attention_mask = True,  
                        return_tensors = 'tf',     
                   )
    
    # Add the encoded sequence to the input list
    input_ids.append(encoded_dict['input_ids'])
    
    # Add the attention mask to the attention mask list
    attention_masks.append(encoded_dict['attention_mask'])

    # Add the token type IDs to the token type ID list
    token_type_ids.append(encoded_dict['token_type_ids'])

# Convert the lists to tensors
input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)
token_type_ids = tf.concat(token_type_ids, axis=0)

# Create labels for the data
labels = []
for label in data['label']:
    if label == 'fake':
        labels.append(1)
    else:
        labels.append(0)

# Convert the labels to a tensor
labels = tf.convert_to_tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


# Model and Callbacks Definition



---



In [ ]:
# Define the BERT model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Dropout
import tensorflow_hub as hub
from tensorflow.keras.models import Model


#preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
seq_length = 256


encoder_inputs = dict(
    input_word_ids=tf.keras.layers.Input(shape=(seq_length,), dtype=tf.int32),
    input_mask=tf.keras.layers.Input(shape=(seq_length,), dtype=tf.int32),
    input_type_ids=tf.keras.layers.Input(shape=(seq_length,), dtype=tf.int32),
)

#input = preprocess('Wow')



bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=False)
input_ids_in = layers.Input(shape=(seq_length,), dtype=tf.int32, name="input_ids")
input_mask_in = layers.Input(shape=(seq_length,), dtype=tf.int32, name="attention_mask")
segment_ids_in = layers.Input(shape=(seq_length,), dtype=tf.int32, name="token_type_ids")

bert_output = bert_layer(encoder_inputs)
pooled_output = bert_output["pooled_output"]
pooled_output = tf.keras.layers.Dropout(0.2)(pooled_output)

# Add additional hidden layers
hidden_layer = Dense(256, activation='relu')(pooled_output)
hidden_layer = Dropout(0.2)(hidden_layer)
hidden_layer = Dense(64, activation='relu')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)
output_layer = Dense(1, activation='sigmoid')(hidden_layer)

# Create a Keras model
model = Model(inputs=encoder_inputs, outputs=output_layer)

# Compile the model
#optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='my_model_weights.h5', 
    save_best_only=True, 
    save_weights_only=True, 
    monitor='val_loss', 
    mode='min', 
    verbose=1
)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    mode='min', 
    verbose=1
)
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=3, 
    min_lr=0.0001, 
    mode='min', 
    verbose=1
)
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='./logs', 
    histogram_freq=1, 
    write_graph=True, 
    write_images=True
)


# Training the model


---



In [ ]:
model.fit(x={'input_word_ids': input_ids, 'input_mask': attention_masks, 'input_type_ids': token_type_ids}, y=labels, epochs=5, batch_size=40, validation_split=0.2, callbacks=[checkpoint_callback, early_stopping_callback, reduce_lr_callback, tensorboard_callback])


Epoch 1/5
898/898 [==============================] - ETA: 0s - loss: 0.3563 - accuracy: 0.8439
Epoch 1: val_loss improved from inf to 0.29232, saving model to my_model_weights.h5
898/898 [==============================] - 908s 993ms/step - loss: 0.3563 - accuracy: 0.8439 - val_loss: 0.2923 - val_accuracy: 0.8716 - lr: 0.0010
Epoch 2/5
898/898 [==============================] - ETA: 0s - loss: 0.2961 - accuracy: 0.8761
Epoch 2: val_loss improved from 0.29232 to 0.22592, saving model to my_model_weights.h5
898/898 [==============================] - 889s 991ms/step - loss: 0.2961 - accuracy: 0.8761 - val_loss: 0.2259 - val_accuracy: 0.9053 - lr: 0.0010
Epoch 3/5
898/898 [==============================] - ETA: 0s - loss: 0.2892 - accuracy: 0.8798
Epoch 3: val_loss improved from 0.22592 to 0.22457, saving model to my_model_weights.h5
898/898 [==============================] - 892s 993ms/step - loss: 0.2892 - accuracy: 0.8798 - val_loss: 0.2246 - val_accuracy: 0.9060 - lr: 0.0010
Epoch 4/5
8

# Saving the model


---



In [ ]:
model.save('model.h5')

In [ ]:
import tensorflow as tf

# Load the trained model from its weights
model = tf.keras.models.load_model('my_model_weights.h5')

In [ ]:
!cp model.h5 drive/MyDrive/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
while True:pass